- Embed pre-analysis conditions of elegibility in the score calculatoin
- Build a score card that reflects the payment quality of a customer, based in past history

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_excel("../tabelas/dataset_variaveis_completo.xlsx")

In [4]:
df.columns

Index(['cnpj', 'segmento', 'valor_divida', 'total_divida', 'composicao_divida',
       'fat_medio', 'prop_divida', 'quantidade_cheques', 'segmento_principal',
       'composicao_principal', 'tempo_medio', 'idade_maxima'],
      dtype='object')

In [5]:
df.shape

(983, 12)

In [11]:
df["cnpj"] = df.apply(lambda x :str(x["cnpj"]), axis=1)

In [13]:
df["cnpj"] = df.apply(lambda x : "0" + x["cnpj"] if len(x["cnpj"])==13 else
                      ("00" + x["cnpj"] if len(x["cnpj"])==12 else x["cnpj"]), axis=1)

In [15]:
df.drop(columns=["valor_divida", "total_divida", "fat_medio"], axis=1, inplace=True)

In [17]:
df.shape

(983, 9)

In [21]:
df["segmento_principal"].unique().tolist()

['processos', 'outros', 'credito', 'infra']

In [22]:
df.groupby(["segmento_principal"]).count()

cnpj  segmento  composicao_divida  prop_divida  \
segmento_principal                                                   
credito              207       207                207          207   
infra                 17        17                 17           17   
outros               656       656                656          656   
processos            103       103                103          103   

                    quantidade_cheques  composicao_principal  tempo_medio  \
segmento_principal                                                          
credito                            207                   207          207   
infra                               17                    17           17   
outros                             656                   656          656   
processos                          103                   103          103   

                    idade_maxima  
segmento_principal                
credito                      207  
infra                         17  
outros                       656  
processos                    103

In [25]:
df[df['segmento_principal']=="infra"]["cnpj"].unique().tolist().__len__()

15

In [27]:
df_infra = df[df['segmento_principal']=="infra"][["cnpj", "prop_divida", "quantidade_cheques", 
                                                 "composicao_principal", "tempo_medio", "idade_maxima"]]

In [31]:
df_infra.shape

(15, 6)

In [30]:
df_infra.drop_duplicates(inplace=True)

In [32]:
df_infra

cnpj  prop_divida  quantidade_cheques  composicao_principal  \
83   03175996000164     0.001078                   0              1.000000   
116  03847026000168     0.025702                   0              0.523699   
322  09202523000195     0.025332                   1              1.000000   
433  11091767000190     0.030607                   0              1.000000   
445  11293987000104     0.001296                   0              1.000000   
451  11326484000180     0.002646                   0              1.000000   
610  17214082000170     0.039981                   0              1.000000   
621  17571404000138     0.053724                   1              1.000000   
628  17910389000105     0.003872                   0              1.000000   
670  19240348000184     0.018463                   0              1.000000   
673  19274088000168     0.000793                   0              1.000000   
691  20124081000149     0.099515                   5              1.000000   
714  20687292000190     0.092050                   0              1.000000   
948  69245405000277     0.034298                   0              0.601341   
971  83535385000119     0.005157                   0              1.000000   

     tempo_medio  idade_maxima  
83      0.000000           270  
116   418.000000           953  
322    46.000000          1121  
433     0.000000           499  
445     0.000000            43  
451     0.000000           203  
610    30.333333           269  
621    30.333333          1471  
628     0.000000          1117  
670     0.000000          1760  
673     0.000000           757  
691     0.000000           785  
714    85.400000           626  
948   779.000000           838  
971     0.000000           387

In [35]:
df[df['cnpj']=='69245405000277']

cnpj segmento  composicao_divida  prop_divida  \
948  69245405000277    infra           0.601341     0.034298   
949  69245405000277   outros           0.398659     0.034298   

     quantidade_cheques segmento_principal  composicao_principal  tempo_medio  \
948                   0              infra              0.601341        779.0   
949                   0              infra              0.601341        779.0   

     idade_maxima  
948           838  
949           838

- for those who pass the elegibility conditions, calculate score
- otherwise, returns not defined

In [36]:
df

cnpj   segmento  composicao_divida  prop_divida  \
0    00178663000281    credito           0.226849    75.183043   
1    00178663000281      infra           0.299252    75.183043   
2    00178663000281     outros           0.151041    75.183043   
3    00178663000281  processos           0.322858    75.183043   
4    00225356000123     outros           1.000000     0.014977   
5    00265954000207     outros           1.000000     0.004024   
6    00272073000302     outros           1.000000     0.070784   
7    00437101000124    credito           0.972151     1.436143   
8    00437101000124     outros           0.027849     1.436143   
9    00661205000118    credito           1.000000     1.232820   
10   00748390000182     outros           1.000000    14.144216   
11   00806759000166    credito           0.679959    45.587042   
12   00806759000166     outros           0.003187    45.587042   
13   00806759000166  processos           0.316854    45.587042   
14   00852832000136     outros           1.000000     0.199687   
15   01056417000139     outros           1.000000     0.021039   
16   01097996000168     outros           1.000000     0.201993   
17   01136501000162     outros           0.949315    10.570655   
18   01136501000162  processos           0.050685    10.570655   
19   01234060000131     outros           1.000000     0.008618   
20   01238137000141     outros           1.000000     0.099454   
21   01344355000160     outros           1.000000     0.569628   
22   01508890000100      infra           0.004588     6.045846   
23   01508890000100     outros           0.995412     6.045846   
24   01509450000177     outros           1.000000     0.014929   
25   01541647000193     outros           1.000000     0.005274   
26   01616681000180     outros           1.000000     0.671589   
27   01623866000111    credito           0.986735    15.074666   
28   01623866000111     outros           0.013265    15.074666   
29   01753779000189    credito           0.962720    87.102691   
..              ...        ...                ...          ...   
953  71702716000189     outros           1.000000   274.552087   
954  72442098000148     outros           1.000000     0.000536   
955  72673403000102     outros           0.202828     0.930389   
956  72673403000102  processos           0.797172     0.930389   
957  73019523000153     outros           1.000000     0.000000   
958  74196221000113     outros           1.000000     0.892610   
959  74196221000113  processos           0.000000     0.892610   
960  74381674000110    credito           0.224391    15.055655   
961  74381674000110     outros           0.775609    15.055655   
962  74686692000100     outros           1.000000     0.022799   
963  76327030000179     outros           1.000000     0.003323   
964  79896528000140     outros           1.000000    11.352728   
965  79905501000177    credito           0.856150    10.965779   
966  79905501000177     outros           0.143850    10.965779   
967  79927778000109     outros           1.000000     0.008293   
968  81345613000162     outros           1.000000     0.022644   
969  81576472000199     outros           1.000000     0.026058   
970  81780819000110     outros           1.000000     0.845340   
971  83535385000119      infra           1.000000     0.005157   
972  83816330000187    credito           1.000000     1.590096   
973  85193449000176     outros           1.000000     0.752760   
974  86692381000132    credito           0.111927  1084.321843   
975  86692381000132      infra           0.000514  1084.321843   
976  86692381000132     outros           0.887559  1084.321843   
977  86692381000132  processos           0.000000  1084.321843   
978  93886505000186     outros           1.000000     0.158019   
979  96364815000129     outros           1.000000     0.081023   
980  97543885000107     outros           1.000000     0.870692   
981  97546276000101    credito           0.761256     3